# EXPERIMENTING

In [1]:
import dolfinx, sys, petsc4py
print("dolfinx version:", dolfinx.__version__)
print("dolfinx path:", dolfinx.__file__)
print("Python:", sys.version)
print("PETSc4py:", petsc4py.__version__)

dolfinx version: 0.9.0
dolfinx path: /srv/conda/envs/notebook/lib/python3.11/site-packages/dolfinx/__init__.py
Python: 3.11.14 | packaged by conda-forge | (main, Oct 13 2025, 14:09:32) [GCC 14.3.0]
PETSc4py: 3.24.0


Bring in needed dependencies

In [2]:
from dolfinx import log, default_scalar_type
from dolfinx.fem.petsc import NonlinearProblem
import pyvista
import numpy as np
import ufl

from mpi4py import MPI
from dolfinx import fem, mesh, plot

from dolfinx.nls.petsc import NewtonSolver
from petsc4py import PETSc

from dolfinx.io import XDMFFile
import traceback

Define the domain

In [3]:
L = 20.0
domain = mesh.create_box(
    MPI.COMM_WORLD, [[0.0, 0.0, 0.0], [L, 4, 4]], [20, 4, 4], mesh.CellType.hexahedron
)
V = fem.functionspace(domain, ("Lagrange", 2, (domain.geometry.dim,)))

Create two python functions for determining the facets to apply boundary conditions to

In [4]:
def left(x):
    return np.isclose(x[0], 0)


def right(x):
    return np.isclose(x[0], L)


fdim = domain.topology.dim - 1
left_facets = mesh.locate_entities_boundary(domain, fdim, left)
right_facets = mesh.locate_entities_boundary(domain, fdim, right)

Next, create a  marker based on these two functions

Concatenate and sort the arrays based on facet indices. Left facets marked with 1, right facets with two

In [5]:
marked_facets = np.hstack([left_facets, right_facets])
marked_values = np.hstack([np.full_like(left_facets, 1), np.full_like(right_facets, 2)])
sorted_facets = np.argsort(marked_facets)
facet_tag = mesh.meshtags(
    domain, fdim, marked_facets[sorted_facets], marked_values[sorted_facets]
)

Create a function for supplying the boundary condition on the left side, which is fixed.

In [6]:
u_bc = np.array((0,) * domain.geometry.dim, dtype=default_scalar_type)

To apply the boundary condition, we identity the dofs located on the facets marked by the `MeshTag`.

In [7]:
left_dofs = fem.locate_dofs_topological(V, facet_tag.dim, facet_tag.find(1))
bcs = [fem.dirichletbc(u_bc, left_dofs, V)]

Next, we define the body force on the reference configuration (`B`), and nominal (first Piola-Kirchhoff) traction (`T`).

In [8]:
B = fem.Constant(domain, default_scalar_type((0, 0, 0)))
T = fem.Constant(domain, default_scalar_type((0, 0, 0)))

Define the test and solution functions on the space $V$

In [9]:
v = ufl.TestFunction(V)
u = fem.Function(V)

Define kinematic quantities used in the problem

In [10]:
# Spatial dimension
d = len(u)

# Identity tensor
I = ufl.variable(ufl.Identity(d))

# Deformation gradient
F = ufl.variable(I + ufl.grad(u))

# Right Cauchy-Green tensor
C = ufl.variable(F.T * F)

# Invariants of deformation tensors
Ic = ufl.variable(ufl.tr(C))
J = ufl.variable(ufl.det(F))

Define the elasticity model via a stored strain energy density function $\psi$,
and create the expression for the first Piola-Kirchhoff stress:

Elasticity parameters

In [11]:
E = default_scalar_type(1.0e4)
nu = default_scalar_type(0.3)
mu = fem.Constant(domain, E / (2 * (1 + nu)))
lmbda = fem.Constant(domain, E * nu / ((1 + nu) * (1 - 2 * nu)))

Stored strain energy density (compressible neo-Hookean model)

In [12]:
psi = (mu / 2) * (Ic - 3) - mu * ufl.ln(J) + (lmbda / 2) * (ufl.ln(J)) ** 2

1st PK stress

In [13]:
P = ufl.diff(psi, F)

Cauchy stress

In [14]:
# --- Compute Cauchy stress symbolically ---
J = ufl.det(F)
sigma = (1.0 / J) * P * F.T  # σ = (1/J) * P * Fᵀ

```{admonition} Comparison to linear elasticity
To illustrate the difference between linear and hyperelasticity,
the following lines can be uncommented to solve the linear elasticity problem.
```

In [15]:
# P = 2.0 * mu * ufl.sym(ufl.grad(u)) + lmbda * ufl.tr(ufl.sym(ufl.grad(u))) * I

Define the variational form with traction integral over all facets with value 2.
We set the quadrature degree for the integrals to 4.

In [16]:
metadata = {"quadrature_degree": 4}
ds = ufl.Measure("ds", domain=domain, subdomain_data=facet_tag, metadata=metadata)
dx = ufl.Measure("dx", domain=domain, metadata=metadata)

Define the residual of the equation (we want to find u such that residual(u) = 0)

In [17]:
residual = (
    ufl.inner(ufl.grad(v), P) * dx - ufl.inner(v, B) * dx - ufl.inner(v, T) * ds(2)
)

As the varitional form is non-linear and written on residual form,
we use the non-linear problem class from DOLFINx to set up required structures to use a Newton solver.

In [18]:
# 1️⃣ Define nonlinear problem
problem = NonlinearProblem(residual, u, bcs=bcs)

# 2️⃣ Configure PETSc options globally
opts = PETSc.Options()
opts["snes_type"] = "newtonls"
opts["snes_linesearch_type"] = "basic"
opts["ksp_type"] = "preonly"
opts["pc_type"] = "lu"
opts["pc_factor_mat_solver_type"] = "petsc"
# You can add monitors if desired:
# opts["snes_monitor"] = ""

# 3️⃣ Create solver and tune tolerances
solver = NewtonSolver(u.function_space.mesh.comm, problem)
solver.rtol = 1e-8
solver.atol = 1e-8
solver.stol = 1e-8
solver.max_it = 50
solver.report = True
solver.convergence_criterion = "incremental"

# 4️⃣ Solve
n_iter, converged = solver.solve(u)
print(f"Converged: {converged}, iterations = {n_iter}")


Converged: True, iterations = 2


Finally, we solve the problem over several time steps, updating the z-component of the traction

In [19]:
# --- Open XDMF file once and keep it open ---
with XDMFFile(domain.comm, "results_hyperelastic.xdmf", "w") as xdmf:
    xdmf.write_mesh(domain)

    tval0 = -1.5

    # --- Loop over load steps ---
    for n in range(1, 20):
        print(f"\n=== Load step {n} ===")
        T.value[2] = n * tval0

        try:
            n_iter, converged = solver.solve(u)
            print(f"Step {n}: {'✅ converged' if converged else '❌ not converged'} in {n_iter} iterations")
        except Exception as e:
            print(f"❌ Solver crashed at step {n}: {e}")
            traceback.print_exc()
            break

        # --- Write displacement field (interpolated to linear space for ParaView) ---
        V_out = fem.functionspace(domain, ("Lagrange", 1, (domain.geometry.dim,)))
        u_out = fem.Function(V_out)
        u_out.interpolate(u)
        u_out.name = "Displacement"
        xdmf.write_function(u_out, t=float(n))  # n used as time value
        print(f"📤 Saved step {n} to XDMF (time = {n})")
        
        # --- Write Cauchy stress ---
        W_out = fem.functionspace(domain, ("Lagrange", 1, (domain.geometry.dim, domain.geometry.dim)))
        v_sig = ufl.TestFunction(W_out)
        tau_sig = ufl.TrialFunction(W_out)
        a_proj = ufl.inner(tau_sig, v_sig) * ufl.dx
        L_proj = ufl.inner(sigma, v_sig) * ufl.dx
        
        problem_proj = fem.petsc.LinearProblem(a_proj, L_proj)
        sigma_out = problem_proj.solve()
        sigma_out.name = "Cauchy_stress"
        
        xdmf.write_function(sigma_out, t=float(n))
        print(f"📤 Saved Cauchy stress step {n}")

        # --- Optional: safety check ---
        if np.isnan(u.x.array).any():
            print("⚠️ NaN detected in solution, stopping loop.")
            break

print("✅ Simulation completed successfully.")



=== Load step 1 ===
Step 1: ✅ converged in 4 iterations
📤 Saved step 1 to XDMF (time = 1)
📤 Saved Cauchy stress step 1

=== Load step 2 ===
Step 2: ✅ converged in 4 iterations
📤 Saved step 2 to XDMF (time = 2)
📤 Saved Cauchy stress step 2

=== Load step 3 ===
Step 3: ✅ converged in 4 iterations
📤 Saved step 3 to XDMF (time = 3)
📤 Saved Cauchy stress step 3

=== Load step 4 ===
Step 4: ✅ converged in 4 iterations
📤 Saved step 4 to XDMF (time = 4)
📤 Saved Cauchy stress step 4

=== Load step 5 ===
Step 5: ✅ converged in 4 iterations
📤 Saved step 5 to XDMF (time = 5)
📤 Saved Cauchy stress step 5

=== Load step 6 ===
Step 6: ✅ converged in 4 iterations
📤 Saved step 6 to XDMF (time = 6)
📤 Saved Cauchy stress step 6

=== Load step 7 ===
Step 7: ✅ converged in 4 iterations
📤 Saved step 7 to XDMF (time = 7)
📤 Saved Cauchy stress step 7

=== Load step 8 ===
Step 8: ✅ converged in 4 iterations
📤 Saved step 8 to XDMF (time = 8)
📤 Saved Cauchy stress step 8

=== Load step 9 ===
Step 9: ✅ converged